# 16. Ch 03. Preprocessing - 08. 실습 형태소 분석기를 활용한 분절하기

In [ ]:
<mecab 맥 설치>
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

-품사까지 출력
echo “아버지가 방에서 나오신다.” | mecab


-세그멘테이션 결과만 출력
echo “아버지가 방에서 나오신다.” | mecab -O wakati

head review.sorted.uniq.refined.tsv

cut -f2 ./review.sorted.uniq.refined.tsv | mecab -O wakati | head -n 2

 

cut -f2 ./review.sorted.uniq.refined.tsv | mecab -O wakati > ./review.sorted.uniq.refined.text.tok

head -n 2 ./review.sorted.uniq.refined.text.tok

paste review.sorted.uniq.refined.label review.sorted.uniq.refined.text.tok > review.sorted.uniq.refined.tok.tsv

코퍼스를 합쳐서 처리하진 않는다. 컬럼 각각의 내용을 분리해서 처리한 뒤 다시 합쳐주는 식으로 셀 스크립트를 작성해 처리하면 쉽게 처리할 수 있다.

# 17. Ch 03. Preprocessing - 09. 분절 길이에 따른 장단점

In [ ]:
형태소 분석기에 따라 분절이 다르게 될 수 있다.
분절의 형태에 따른 특성

<토큰 평균 길이에 따른 성격과 특징>
짧을 수록
-vocabulary 크기 감소
희소성 문제 감소
-OoV(Out of Vocabulary, <UNK>로 치환>가 줄어듬
-(기억해야 하는)Sequence의 길이가 길어짐
모델의 부담 증가
-극단적 형태: character 단위

길 수록
-Vocabulary 크기 증가
희소성 문제 증대
-OoV가 늘어남
-Sequence의 길이가 짧아짐
모델의 부담 감소

-> 토큰의 길이에 따른 트레이드오프 존재, 어느 정도의 토큰 길이가 괜찮겠다는 기준이 있어야 한다.
<UNK>는 언어모델에서 굉장히 치명적으로 작용, UNK 다음 단어는 뭐가 나와도 된다

<정보량에 따른 이상적 형태>
-빈도가 높을 경우 하나의 token으로 나타내고,(문장이라고 할지라도)
-빈도가 낮을 경우 더 잘게 쪼개어, 각각 빈도가 높은 토큰으로 구성한다
-> 압축 알고리즘?과 굉장히 유사
정보이론과도 굉장히 밀접한 관계가 있다.

# 18. Ch 03. Preprocessing - 10. 서브워드 분절

In [ ]:
<단어보다 더 작은 의미 단위 : 서브워드>
-많은 언어들에서, 단어는 더 작은 의미 단위들이 모여 구성됨
영어: Concentrate / con(together) + cent(center) + ate(make)
한국어: 집중 /  모을 집 + 가운데 중
-따라서 이러한 작은 의미 단위로 분절할 수 있다면 좋을 것
-하지만 이를 위해선 언어별 서브워드 사전이 존재해야 할 것

<Byte Pair Encoding(BPE) 알고리즘>
-압축 알고리즘을 활용해 서브워드 세그멘테이션을 적용
Sennrich at el., 2015
-학습 코퍼스를 활용해 BPE 모델을 학습 후, 학습/테스트 코퍼스에 적용
-장점:
희소성을 통계에 기반해 효과적으로 낮출 수 있다.
언어별 특성에 대한 정보 없이, 더 작은 의미 단위로 분절할 수 있다.(데이터 드리븐)
OoV를 없앨 수 있다. (seen character로만 구성될 경우)
-단점:
학습 데이터 별로 BPE 모델도 생성됨

<BPE Training & Applying>
-트레이닝
1)단어 사전 생성(빈도 포함)
2)캐릭터 단위로 분절 후, pair별 빈도 카운트
3)최빈도 페어를 골라, merge 수행
4)페어별 빈도 카운트 업데이트
5)3번 과정 반복
-어플라잉
1)각 단어를 캐릭터 단위로 분절
2)단어 내에서 학습 과정에서 머지에 활용된 페어의 순서대로 머지 수행

트레이닝 셋의 역할이 굉장히 중요. 테스트셋과 다른 분야의 말뭉치였다고 한다면 테스트셋의 최적의 분절은 트레이닝셋의 분절과 다를 수 있다.
실제로 같은 도메인이어야 한다는 것이 중요. 
그게 아니어도 가장 중요한 것은 테스트 과정에서 트레이닝의 모델을 가지고 분절을 수행한다는 것.

<subword segmentation Modules>
세그멘테이션 직접 구현할 필요 없다.
-subword-nmt
-WordPiece
BPE 버전 업그레이드. 현재 사용 불가?
-SentencePiece(가장 최신) 혹은 서브워드 nmt 쓰면 될 듯

<OoV가 미치는 영향>
-입력 데이터에 OoV가 발생할 경우, <UNK> 토큰으로 치환해 모델에 입력
예) 나는 학교에 가서 밥을 먹었다. -> 나 는 <UNK> 에 가 서 <UNK> 을 먹 었 다 . 
특히 이전 단어들을 기반으로 다음 단어를 예측하는 태스크에서 치명적
예) 자연어 생성 태스크
분류 태스크는 생각보다 치명적이지는 않음. 보통 중요하지 않은 단어인 경우가 많기 때문에.
다른 단어들로 충분히 분류 가능.
-어쨌든 모르는 단어지만, 알고 있는 서브워드들을 통해 의미를 유추해 볼 수 있음.
예) 버카충

<요약>
-BPE 압축 알고리즘을 통해 통계적으로 더 작은 의미 단위(서브 워드)로 분절 수행
-BPE를 통해 OoV를 없앨 수 있으며, 이는 성능상 매우 큰 이점으로 작용
-한국어의 경우
띄어쓰기가 제멋대로인 경우가 많으므로, normalization 없이 바로 서브워드 세그멘테이션을 적용하는 것은 위험
따라서 형태소 분석기를 통한 토크나이제이션을 진행한 이후, 서브워드 세그멘테이션을 적용하는 것을 권장.
띄어쓰기를 노멀라이제이션 해주고 나서(표준어로 바꿔주고 나서) 서브워드 세그멘테이션 진행하는 게 낫다는 것.